In [1]:
import numpy as np
import pandas as pd 
import requests
import json
from bs4 import BeautifulSoup, SoupStrainer

In [1348]:
import importlib

In [1388]:
importlib.reload(crawler.overview_data)

<module 'crawler.overview_data' from 'C:\\Users\\Kevin\\Documents\\GitHub\\fifa18-all-player-statistics\\crawler\\overview_data.py'>

In [1352]:
html.update_pickled_overview_htmls(overview_htmls)

In [2]:
from crawler.overview_data import get_overview_data

In [1339]:
%time overview_htmls = html.get_overview_htmls()

Wall time: 16 s


In [3]:
%time y = get_overview_data(from_file=True)

Wall time: 33.4 s


In [923]:
%time overview_htmls = get_overview_htmls(from_file=True)

Wall time: 7.3 s


In [447]:
save_htmls_to_pickle(overview_htmls, 'overview')

In [1359]:
script_strainer = SoupStrainer('script')
script_url = 'https://sofifa.com/player/158023'
script_html = requests.get(script_url).text
script_soup = BeautifulSoup(script_html, 'lxml', parse_only=script_strainer)

In [125]:
player_data_url = 'https://sofifa.com/player/20801'
# skill_names = ['ID', 'crossing', 'finishing', 'heading_accuracy','short_passing', 'volleys', 'dribbling', 'curve',
#                'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility',
#                'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
#                'interceptions', 'positioning', 'vision', 'penalties', 'composure', 'marking', 'standing_tackle',
#                'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']
# headline_attribute_names = ['PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY']
# all_attribute_names = skill_names + headline_attribute_names
player_attribute_dict = {'ID': 20801}

In [495]:
source_code = requests.get(player_data_url)
gk_source_code = requests.get('https://sofifa.com/player/228736')

In [810]:
plain_text = source_code.text
strainer = SoupStrainer(['section', 'script'])
soup = BeautifulSoup(plain_text, 'lxml', parse_only=strainer)
gk_soup = BeautifulSoup(gk_source_code.text, 'lxml', parse_only=strainer)

Wall time: 65.5 ms


In [1298]:
test_IDs = player_overview_data['ID'].sample(50)
player_htmls = get_player_htmls(test_IDs)

In [1320]:
x = parse_player_detailed_data(player_htmls, constants)

In [528]:
def main(from_file=False):
    player_overview_data = get_overview_data(from_file)
    IDs = player_overview_data['ID']
    player_detailed_data = get_player_detailed_data(IDs, from_file)
    merged = player_overview_data.merge(player_detailed_data, on='ID')
    return merged